# Zadanie 3

## Analiza obwodu Elektrycznego


In [2]:
import networkx as nx
import numpy as np

from copy import deepcopy
from collections import deque
from random import randint, random
from pyvis.network import Network

#### Convertion between different graph representations


In [3]:
def to_neibour_representation(edges):
    graph = []
    for v1, v2, r in edges:
        while len(graph) <= max(v1, v2):
            graph.append([])
        graph[v1].append((v2, r))
        graph[v2].append((v1, r))
    return graph

## Fuctions for finding cycles in a graph


In [4]:
def find_cycle_from_edge(graph, a, b):
    n = len(graph)
    visited = [False for _ in range(n)]
    visited[b] = True
    stack = deque()
    road = deque([a])
    return find_cycle(graph, visited, road, stack, initial=a, vertex=b)


def find_cycle_from_vertex(graph, initial):
    visited = [False for _ in graph]
    stack = deque()
    road = deque()
    return find_cycle(graph, visited, road, stack, initial, vertex=initial)


def find_cycle(graph, visited, road, stack, initial, vertex, debug=False):
    if debug:
        print("START")
    returning = False

    while not (vertex == initial and len(road) > 1):

        if not returning and vertex != initial:
            visited[vertex] = True

        if debug:
            print(f"v: {vertex}, returning: {returning}")
            print(f"stack: {stack}")
            print(f"road: {road}")
            print(f"visited: {visited}")

        if returning:
            if len(stack) == 0:
                return None
            next_v = stack[-1]

            if debug:
                print("Trying to get to:", next_v)

            for neib, _ in graph[vertex]:
                if neib == next_v:
                    returning = False
                    break
            # Still returning
            if returning:
                visited[vertex] = False

                road.pop()
                next_v = road[-1]

            else:
                stack.pop()

        else:
            returning = True
            road.append(vertex)
            for neib, _ in graph[vertex]:
                if (not visited[neib]) and (len(road) > 2 or neib != initial):
                    returning = False
                    stack.append(neib)

                    if debug:
                        print(f"Added: {neib}")
            if len(stack) == 0:
                return None
            next_v = stack.pop()

            if returning:
                visited[vertex] = False
                road.pop()
                stack.append(next_v)
                next_v = road[-1]

        vertex = next_v

    if debug:
        print("DONE")
        print("")
    return road

In [5]:
def new_get_n_cycles(edges_origin, neib_rep_origin, n):
    edges = deepcopy(edges_origin)
    neib_rep = deepcopy(neib_rep_origin)
    cycles = []

    for _ in range(n):
        edge_ind = -1
        while True:
            a, b, _ = edges[edge_ind]
            cycle = find_cycle_from_edge(neib_rep, a, b)
            if cycle is not None:
                break
            edge_ind -= 1

        edges.pop(edge_ind)
        for i in range(len(neib_rep[a])):
            if neib_rep[a][i][0] == b:
                neib_rep[a].pop(i)
                break
        for i in range(len(neib_rep[b])):
            if neib_rep[b][i][0] == a:
                neib_rep[b].pop(i)
                break
        cycles.append(cycle)
    return cycles

In [6]:
def verticle_cycle_into_edges(verticle_cycle, edges):
    result = []
    for v in range(len(verticle_cycle)):
        a, b = verticle_cycle[v], verticle_cycle[(v + 1) % len(verticle_cycle)]
        for i in range(len(edges)):
            if (edges[i][0], edges[i][1]) == (a, b):
                result.append((i, 1))
                break
            elif (edges[i][0], edges[i][1]) == (b, a):
                result.append((i, -1))
                break
    return result

## Function that solves the whole electric Circuit

For each edge we need to find its intensity, so we have E unknowns.

I Kirchoff Law gives us V-1 equasions from circut verticles. The last verticle is leary dependent on the rest.

The remaining E-V+1 equasions we need to recieve using II Kirchoff Law. For that finding E-V+1 independent cycles in a graph is required.
To guarrantee that all cycles are independent, we "remove" first edge of each cycle we found from the graph.


In [7]:
def solve(edges, neib_rep=None):
    if neib_rep is None:
        neib_rep = to_neibour_representation(edges)
    e = len(edges)
    v = len(neib_rep)

    # We have e unknowns, so we need e equasions
    matrix = np.zeros(shape=(e, e))
    vector = np.zeros(shape=e)

    # V-1 equasions from I Kirchoff Law
    for i in range(v - 1):
        # print(f'v{i}:')
        for k in range(e):
            # Flowing into vertex
            if edges[k][1] == i:
                # print(f'+ I{k}')
                matrix[i, k] = 1
            # Flowing out of vertex
            elif edges[k][0] == i:
                # print(f'- I{k}')
                matrix[i, k] = -1

    # The rest (E - V + 1) equasions from II Kirchoff Law
    # cycles = get_n_cycles(edges, neib_rep, n=e - v + 1)
    cycles = new_get_n_cycles(edges, neib_rep, n=e - v + 1)
    i = v - 1
    for cycle in cycles:
        edges_in_cycle = verticle_cycle_into_edges(cycle, edges)
        for edge_index, sign in edges_in_cycle:
            # Electromotive force applied: +/- ε
            if edge_index == 0:
                vector[i] = sign * edges[0][2]
            # +/- I * R
            else:
                matrix[i, edge_index] = sign * edges[edge_index][2]
        i += 1

    return np.linalg.solve(matrix, vector)

## Functions for visualization of graphs


In [8]:
BLACK = "000000"
LIGHT_GREEN = (153, 255, 153)
DARK_RED = (133, 11, 11)


def rgb_to_hex(rgb):
    return "%02x%02x%02x" % rgb


def get_edge_color(val, max_val, max_color=DARK_RED, min_color=LIGHT_GREEN):
    factor = abs(val) / abs(max_val)
    return rgb_to_hex(
        tuple(
            [
                int(part[0] * (1 - factor) + part[1] * factor)
                for part in zip(min_color, max_color)
            ]
        )
    )


def show_graph(edges, vertices_number, electric_currents):
    g = Network(directed=True)

    g.add_nodes(
        [i for i in range(vertices_number)],
        title=["Node " + str(i) for i in range(vertices_number)],
        size=[5 for i in range(vertices_number)],
        color=[BLACK for i in range(vertices_number)],
    )

    is_voltage = True
    for (a, b, resistance), current in zip(edges, electric_currents):
        if current < 0:
            a, b = b, a
        message = (
            "U=%.2f" % resistance + "V" if is_voltage else "R=%.2f" % resistance + "Ω"
        )
        message += ", I=%.2fA" % abs(current)

        color = get_edge_color(val=current, max_val=electric_currents[0])
        g.add_edge(a, b, title=message, label=message, color=color)
        is_voltage = False

    g.show("example.html", notebook=False)

# Generating random graphs

They are sanitised: each vertex is connected to at least two other ones.


In [10]:
def generate_random_graph(n, electromotorive_force=10, resistance_bounds=(1, 10)):
    get_random_resistance = lambda: randint(resistance_bounds[0], resistance_bounds[1])
    edges = []
    for i in range(n - 1):
        connected = randint(i + 1, n - 1) if i != 0 else n - 1
        edges.append((i, connected, get_random_resistance()))

        for j in range(i + 1, n):
            if i != j and j != connected and random() < 0.2:
                edges.append((i, j, get_random_resistance()))

    neib_rep = to_neibour_representation(edges)

    # Sanitisation
    for i in range(n):
        while len(neib_rep[i]) < 2:
            new_friend = randint(0, n - 1)
            if new_friend != i and (
                len(neib_rep[i]) == 0 or neib_rep[i][0][0] != new_friend
            ):
                edges.append((i, new_friend, get_random_resistance()))
                neib_rep[i].append((new_friend, -1))
                neib_rep[new_friend].append((i, -1))

    edges[0] = edges[0][0], edges[0][1], electromotorive_force
    return edges

In [130]:
def generate_bridge_random_graph(n, electromotorive_force=10):
    if n < 6:
        raise ("Sorry amigo, 6 at least.")
    n1 = (n + 1) // 2
    n2 = n // 2

    edges1 = generate_random_graph(n1)
    edges2 = generate_random_graph(n2)
    for i in range(len(edges2)):
        a, b, r = edges2[i]
        edges2[i] = a + n1, b + n1, r
    return (
        [(0, n1, electromotorive_force)]
        + edges1
        + edges2
        + [(randint(1, n1 - 1), randint(n1 + 1, n - 1), randint(1, 10))]
    )

In [12]:
def generate_petersen_graph(electromotorive_force=10, resistance_bounds=(1, 10)):
    get_random_resistance = lambda: randint(resistance_bounds[0], resistance_bounds[1])
    edges = []
    for i in range(5):
        new_edges = [
            (i, (i + 1) % 5, get_random_resistance()),
            (i, i + 5, get_random_resistance()),
            (i + 5, ((i + 2) % 5) + 5, get_random_resistance()),
        ]
        edges += new_edges

    edges[0] = edges[0][0], edges[0][1], electromotorive_force
    return edges

In [48]:
def generate_grid_graph(size, electromotorive_force=10, resistance_bounds=(1, 10)):
    get_random_resistance = lambda: randint(resistance_bounds[0], resistance_bounds[1])
    x, y = size
    N = x * y
    # sem = ((x+1) // 2, (y+1) // 2)
    sem = (x // 2, (y - 1) // 2)
    sem_ind = None
    sem_i = None

    edges = []
    for i in range(N):
        y0 = i // x
        x0 = i % x
        if x0 < x - 1:
            edges.append((i, i + 1, get_random_resistance()))
        if y0 < y - 1:
            if (x0, y0) == sem:
                sem_ind, sem_i = len(edges), (i, i + x)
            edges.append((i, i + x, get_random_resistance()))
    edges.pop(sem_ind)
    edges = [(sem_i[0], sem_i[1], electromotorive_force)] + edges
    return edges

# Code to generate examples

N variable determines number of verticles.
Graph visualisation is saved as _example.html_


#### Random graph


In [70]:
N = 12

random_edges = generate_random_graph(N)
currents = solve(edges=random_edges)
show_graph(random_edges, N, currents)

example.html


#### Bridge random graph


In [129]:
N = 7

random_edges = generate_bridge_random_graph(N)
currents = solve(edges=random_edges)
show_graph(random_edges, N, currents)

example.html


#### A cubic graph: Peterson graph


In [62]:
petersen_graph_edges = generate_petersen_graph()
currents = solve(edges=petersen_graph_edges)
show_graph(petersen_graph_edges, 10, currents)

example.html


#### Grid graph


In [65]:
size = (8, 4)

grid_edges = generate_grid_graph(size)
# show_graph(grid_edges, size[0] * size[1], [1 for _ in grid_edges])
currents = solve(edges=grid_edges)
show_graph(grid_edges, size[0] * size[1], currents)

example.html


In [173]:
## Testing correctness of the sollution

In [174]:
def check_first_kirchoff_law(edges, currents, verticle_number, eps=10**-8):
    results = [0 for _ in range(verticle_number)]

    for (a, b, _), curr in zip(edges, currents):
        results[a] -= curr
        results[b] += curr
    # return results
    for current in results:
        # print(current)
        if abs(current) > eps:
            return False
    return True

In [175]:
SMALL = 6
MID = 20
BIG = 40

print("Preparing tests...")
tests = []

for number in (SMALL, MID, BIG):
    for generate_function in (generate_random_graph, generate_bridge_random_graph):
        tests.append((generate_function(number), number))
    tests.append(
        (
            generate_grid_graph(size=(number // 2, number // 2)),
            (number // 2) * (number // 2),
        )
    )
(generate_petersen_graph(), 10)

print("Solving...")
currents = [solve(edges) for (edges, N) in tests]

print("ok")
i = 1
for (edges, N), result in zip(tests, currents):
    print(f"Test {i}:", end=" ")
    if check_first_kirchoff_law(edges, result, verticle_number=N):
        print("PASSED")
    else:
        print("FAILED")
        break
    i += 1

Preparing tests...
Solving...
ok
Test 1: PASSED
Test 2: PASSED
Test 3: PASSED
Test 4: PASSED
Test 5: PASSED
Test 6: PASSED
Test 7: PASSED
Test 8: PASSED
Test 9: PASSED


# Examples

## Basic graph

![title](img/small_graph.png)

## Random graph

![title](img/random_graph.png)

## Bridge graph

![title](img/bridge_graph.png)

## Peterson graph (Cubic)

![title](img/bridge_graph.png)

## Grid graph

![title](img/grid.png)
